## Model Features

### Changeover Times
- estimated by cell

## Changeover Time Development

### Total Units by Split

- A production system has $i$ products
- There is a horizon $T_{horiz}$ over which the system operates
- For each product, the fraction of the total quantity produced is $X_i$
- We need to find the time to operate each configuration $T_i$ and the total number of periods $N_{periods}$ to run

#### Variables
__System__
- Total horizon time $T_{horizon}$
- Total production quantity $Q_{system}$
- Production Periods $N_{periods}$

__Products__
- Run time per period $T_i$
- Product run rate $\lambda_i$
- Product quantity split $X_i$

#### Inputs
- Horizon time $T_{horiz}$
- Product (class) mix splits $X_i$
- Total Quantity $Q_{system}$

#### Formulation

The total production window is constrained by the class production times and any additional changeover times. This is formulated as a posynomial inequality. 
Is there sufficient pressure on $N_{periods}$ and $T_i$ for this constraint to be tight? Yes, it appears with holding costs, and changeout time, there is enough pressure. 
$$ T_{horiz} \geq N_{periods} \sum T_i + N_{periods} T_{changeover} $$

If the factory is not operating all the time, then we need to consider the downtime $T_{downtime}$ (e.g. nights, weekends, holidays)
$$ T_{horiz} \geq N_{periods} \sum T_i + N_{periods} T_{changeover} + T_{downtime}$$

The total production volume $Q_{system}$ is determined by $T_{horizon}$ and $\lambda_{system}$. 
$$ Q_{system} = T_{horizon}\lambda_{system} $$

System rate should be left as a free variable (as we currently do with systems) so that it may be maximized.

There are several, linearlly-dependant constraints which describe the relationship between the system and the class splits:
$$ N_{periods} T_i \lambda_i = Q_i = X_i Q_{system} $$
$$ \lambda_i = X_i \lambda_{system} $$
Selecting the correct constraint here could be tricky.

The production time for each class $T_i$ should be more than the time it takes for the class to occupy the line based on the flow time and the inventory in the line
$$ T_i \geq L_i W_i $$
applying Little's Law $L = \lambda W$ gives an equivalent formulation which may substituted for convenience:
$$ T_i \geq \lambda_i W_i^2 $$


For any shared cells, the workstation counts should be the same across the classes. One could make the argument that we just pay for the maximum number of workstations for any class' production line so that, if the optimization were to find it advantageous to only partially utilize the stations, that would still be permitted (for example, if there is a lot of recurring cost associated with a workstation).


### Estimating Changeover Time

- (Conservative) empty the line of all inventory, changeover all lines (limited by max cell changeover time), restart line
- (Better scenario) progressive changeover by cell: sum the cell changeover times
  - Do we have to determine which is the faster product?
  - A faster product will have to wait behind the slower product and the changeover

#### Approaches to Changeover Time
Roughly in order of complexity to implement/research
1. "Flow-out" changeover. (Empty the line and find the max changeover)
2. Simple progressive (find buildup along the line)
3. Fully separate lines for products + blanks
4. Separate multiproduct lines each with product + blanks

__Variables__
- Changeover time at each cell $t_{c,i}$
- Total changeover time for each phase $T_c$
- The number of products in the system $N_{prods}$

##### 1) Flow-Out Changeover

With "flow-out" have to empty the line of the product. Each phase will have a changeout for each product. Will look for the max changeover in the path of the product $i$ since not all resources may be shared by all products.
$$ T_c \geq \sum_{i=1}^{n_{products}}{L_i W_i + \max(t_{c,j} \forall j \in path_i)} $$
Perhaps, we also have to consider the amount of time the line has to fill back up as well? Since we are measuring the changes to the production time, we are esentially calculating the time that the stations are empty. This can be thought about as the time when the line is *not* producing. As the line empties, there are still parts produced. But, as the line fills back up 

##### 2) Simple Progressive

Have to wait all extra for each changeover. The last changeover will block the whole system which is why the waiting time is a function of $i$.

$$ T_c \geq \mathop{\sum_{products}\sum_{i=1}^{n_{cells}} i t_{c,i} \sqrt{m_i}} $$

Enumerate from the list of cells in the line for each product (if a shared resource).

The first part through any workstation will have to experience the changeover. Parts coming behind will experience changeover with likelihood $\sqrt{m_i}$
$$ T_c \geq \mathop{\sum_{products}\sum_{i=1}^{n_{cells}} t_{c,i}\left[\sqrt{m_i}(i-1)\right] + m_i} $$

But, I'm not convinced extra flow time equates to "changeover time" or time not available for production...

##### 2A) Revision to Simple Progressive with Parallel Workstations

If there is a new part arriving to the cell, 

#### 2B) Additional time due to a decreased rate accompanying a progressive changeover

$$ \omega* $$

### Relevant Costs

- assume we are trying to minimize average unit cost over the period
- recurring costs will continue even during changeover
- holding costs of finished gods (assuming full mix at dispatch)(should be a decreasing function of $N$)
  - assume that there is a shipment at the end of every cycle
  - the total number of shipments can be determined by the outbound capacity (outbound logistics) which can also put some pressure on $N$


Holding costs are determined for each product as the function of its run time, the run time of the other products, holding value $d_i$, holding rate $h$ with units #\frac{%}{time}$ and period production quantity $Q_i/N_{periods}$ and the holding time. There is an additional term $\nu$ that represents the ratio of available time to average run time to account for hours when the system is not producing (e.g. weekends, nights, holidays, etc.)

$$ cost_{holding} \geq h d_i \frac{Q_i}{N_{periods}} \left( \frac{T_i}{2} + \sum_{j \neq i} T_j \right) \nu$$

There may be additional costs based on $N_{periods}$ that are related to outbound transportation costs. 

$$ f(\text{transportation cost}) \Downarrow N_{periods} $$
$$ f(\text{carrying costs}) \Uparrow N_{periods} $$

#### Recurring Costs

Recurring costs for each product can use the production time 

#### Shared Resource Costs

- Non-recurring capital costs are calculated by resources
- For shared resources, the count is the maximum across all the classes sharing the resource
- Recurring costs of resources are priced per class (since they are running phased)
  - some resources may be idle during some phases of production, this reduces changeover time and reduces recurring costs

### Constraints

$$ T_{horiz} \geq N_{periods} T_{changeover} + N_{periods} \sum T_i \quad \text{total horizon time}$$
$$ X_i Q_{system} = Q_i \quad \forall i \quad \text{product splits} $$
$$ Q_i \leq \lambda_i T_i N_{periods} \quad \forall i \quad \text{product production rate} $$
$$ Q_{sys} = T_{horizon} \lambda_{system} \quad \text{total production quantity}$$
$$ T_i \geq 2 \lambda_i W_i^2 \quad \forall i \quad \text{minimum run time}$$


__Revised Constraint List__
$$ T_{horiz} \geq N_{periods} T_{changeover} + N_{periods} \sum T_i \quad \text{total horizon time}$$
$$ Q_{system} X_i = N_{periods} \lambda_i T_i, \quad \forall i \in Products \quad \text{total production rate} $$
$$ m_i \geq \max(m_{j,i}) \quad \forall i,j \in Products, Cells \quad \text{find maximum station count for costing} $$
$$ T_i \geq 2 L_i W_i, \quad \forall i \in Products \quad \text{set minimum run time} $$

## Test Model

In [2]:
from gpkit import Model, Variable, VectorVariable, units, SignomialsEnabled

ImportError: No module named 'gpkit'

In [ ]:
from collections import OrderedDict as OD
import numpy as np

In [ ]:
from gpx.manufacturing import Process, QNACell, FabLine

#### First Product

In [ ]:
p1_processes = OD()

In [ ]:
# Create Processes
p1_processes['p1 | first process'] = Process()
p1_processes['p1 | second process'] = Process()

In [ ]:
# Create Cells
p1_cells = {
    'First Cell' : QNACell(process=p1_processes['p1 | first process']),
    'Second Cell' : QNACell(process=p1_processes['p1 | second process']),
}

In [ ]:
p1_line = FabLine(cells=list(p1_cells.values()))

In [ ]:
p1_cost = np.sum([c.m for c in p1_line.cells]) + p1_line.L

In [ ]:
p1_line.substitutions.update({
    p1_processes['p1 | first process'].t    : 15*units('min'),
    p1_processes['p1 | second process'].t   : 30*units('min'),
    p1_processes['p1 | first process'].cv   : 0.5,
    p1_processes['p1 | second process'].cv  : 0.25,
    p1_line.lam  : 100*units('count/hr'),
})

##### Quick Test Solve

In [ ]:
m = Model(p1_cost, p1_line)

In [ ]:
print(m.solve(solver='cvxopt').table())

Using solver 'cvxopt'
 for 22 free variables
  in 24 posynomial inequalities.
Solving took 0.0215 seconds.

Optimal Cost
------------
 159.2

Free Variables
--------------
         | FabLine92
       L : 79.43    [count]    Total WIP count
       W : 0.7943   [hr]       Total flow time

         | QNACell184
       W : 16.08    [min]      Total flow time through cell
      Wq : 1.083    [min]      Expected queueing time
  \alpha : 0.07601             (1-rho)
 \lambda : 100      [count/hr] Production rate
     c2a : 0.07142             Arrival coefficient of variation squared
     c2d : 0.2364              Departure coefficient of variation squared
     c2s : 0.25                Process coefficient of variation squared
       m : 27.06    [count]    Number of parallel workstations
     rho : 0.924               Cell utilization
t_{\eta} : 15       [min]      Average cycle time including efficiency

         | QNACell185
       W : 31.57    [min]      Total flow time through cell
      W

#### Second Product

In [ ]:
p2_processes = OD()

In [ ]:
p2_processes['p2 | first process'] = Process()
p2_processes['p2 | second process'] = Process()

In [ ]:
p2_cells = {
    'First Cell'    : QNACell(process=p2_processes['p2 | first process']),
    'Second Cell'    : QNACell(process=p2_processes['p2 | second process']),
}

In [ ]:
p2_line = FabLine(cells=list(p2_cells.values()))

In [ ]:
p2_cost = np.sum([c.m for c in p2_line.cells]) + p2_line.L

In [ ]:
p2_line.substitutions.update({
    p2_processes['p2 | first process'].t    : 45*units('min'),
    p2_processes['p2 | second process'].t   : 30*units('min'),
    p2_processes['p2 | first process'].cv   : 0.25,
    p2_processes['p2 | second process'].cv  : 0.25,
    p2_line.lam  : 150*units('count/hr'),
})

##### Quick Test Solve

In [ ]:
m2 = Model(p2_cost, p2_line)

In [ ]:
print(m2.solve(solver='cvxopt').table())

Using solver 'cvxopt'
 for 22 free variables
  in 24 posynomial inequalities.
Solving took 0.0272 seconds.

Optimal Cost
------------
 384.5

Free Variables
--------------
         | FabLine93
       L : 192.2    [count]    Total WIP count
       W : 1.281    [hr]       Total flow time

         | QNACell186
       W : 46.04    [min]      Total flow time through cell
      Wq : 1.037    [min]      Expected queueing time
  \alpha : 0.02302             (1-rho)
 \lambda : 150      [count/hr] Production rate
     c2a : 0.0625              Arrival coefficient of variation squared
     c2d : 0.0625              Departure coefficient of variation squared
     c2s : 0.0625              Process coefficient of variation squared
       m : 115.2    [count]    Number of parallel workstations
     rho : 0.977               Cell utilization
t_{\eta} : 45       [min]      Average cycle time including efficiency

         | QNACell187
       W : 30.84    [min]      Total flow time through cell
      W

#### System

In [ ]:
# System variables
Qsys = Variable('Q_{system}', 'count', 'system production volume')
Thorizon = Variable('T_{horizon}', 'hrs', 'total production horizon')
lamsys = Variable('\\lambda_{system}', 'count/hr', 'system production rate')
nperiods = Variable('N_{periods}', 'count', 'number of production periods')
davg = Variable('d_{average}', '-', 'average inventory value')
ctot = Variable('c_{total}', '-', 'total cost')

In [ ]:
# system inputs
h = Variable('h', 0.1, '1/hr', 'holding rate')

In [ ]:
# make lists for convenience
sys_lines = [p1_line, p2_line]

# define shared resources as list of lists
num_shared_resources = 2
shared_resources = [
    (p1_line.cells[0], p2_line.cells[0]), 
    (p1_line.cells[1], p2_line.cells[1])
]

# variables for max workstation counts
mmax = VectorVariable(num_shared_resources, 'm_{max}', 'count', 'max workstation count of shared resources')

In [ ]:
# Vector Variables for Products
num_prods = 2
# Q = VectorVariable(num_prods, 'Q', 'count', 'total production quantity')
X = VectorVariable(num_prods, 'X', '-', 'overall production split')
T = VectorVariable(num_prods, 'T', 'hrs', 'production time per period')
cholding = VectorVariable(num_prods, 'c_{holding}', '-', 'holding cost')

In [ ]:
# changeover variables
Tchangeover = Variable('T_{changeover}', 'hrs', 'total changeover time per period')
# tprodchange = VectorVariable(num_prods, 'hrs', 'total product changeover time')
tci = VectorVariable(2, 't_c', [30,20], 'min', 'cell changeover time')

In [ ]:
const = []

In [ ]:
# total system time
const.append(Thorizon >= nperiods*np.sum(T) + nperiods*Tchangeover)

In [ ]:
# calculate holding cost for each product
for i in range(num_prods):
    const.append(cholding[i] >= h*davg/nperiods*Qsys*X[i]*(T[i]/2 + np.sum([T[j] for j in range(num_prods) if j != i])))

In [ ]:
# system production quantity based on product rates
for i in range(num_prods):
    const.append(Qsys*X[i] == sys_lines[i].lam*nperiods*T[i])

In [ ]:
# find maximum of workstation counts
for i,r in enumerate(shared_resources):
    for cell in r:
        const.append(mmax[i] >= cell.m)

In [ ]:
from gpkit.constraints.tight import Tight
from gpkit.constraints.loose import Loose

In [ ]:
# set minimum production times
# ideally do not want to see this as the limiting constraint
# if there is little other pressure on inventory, this will be an active constraint
# as soon as inventory has additional cost, this is not an active constraint
for i in range(num_prods):
    const.append(Loose([T[i] >= 2*sys_lines[i].L*sys_lines[i].W]))

In [ ]:
# calculated different changeover times
Tchangeover_flowout = Variable('T_{change, flow-out}', 'hr', 'full flow-out changeover')
maxtci = Variable('\max{t_c}', 'hr', 'path max changeover')
Tchangeover_progressive = Variable('T_{change, progressive}', 'hr', 'progressive changeover')


## Flow-out Changeover

for i,_ in enumerate(tci):
    const.append(maxtci >= tci[i])

const.append(Tight([Tchangeover_flowout >= np.sum([l.L*l.W + maxtci for l in sys_lines])/units('count')]))

## Progressive Changeover
# const.append(Tight([
#     # Tchangeover_progressive >= np.sum([tci[i]*(i*sl.cells[i].m**0.5 + sl.cells[i].m) for i in range(num_shared_resources) for sl in sys_lines])/units('count'),    # account for 
#     Tchangeover_progressive >= np.sum([tci[i]*(i+1)/sl.L*(sl.cells[i].m)**0.5*units('count')**0.5 for i in range(num_shared_resources) for sl in sys_lines]),    # try average changeover flow time
# ]))

# old progressive approach
const.append(Tchangeover_progressive >= num_prods*np.sum([(i+1)*tci[i] for i in range(num_shared_resources)]))   # we should also see an effect of m

# find the worst case

select_best_case = True

if not select_best_case:
    const.extend([
        Tchangeover >= Tchangeover_progressive,
        Tchangeover >= Tchangeover_flowout,
    ])

if select_best_case:
    # Model the best case with SP
    select_t = VectorVariable(2, 'x_{changeover, select}', '-', 'changeover method selection')

    # set up the selection for the lower cost
    with SignomialsEnabled():
        const.append(np.sum(select_t) >= 1),

    const.append(
        Tchangeover >= select_t[0]*Tchangeover_flowout + select_t[1]*Tchangeover_progressive
)


In [ ]:
## Progressive Changeover 2B
summands = []
for l in sys_lines:
    # for each product
    summands.append(
        np.sum([tci[i]*c.m**0.5 * np.sum([l.cells[j].W*l.cells[j].lam for j in range(i+1)])*units('count')**0.5 for i,c in enumerate(l.cells)])/l.L
    )

const.append(Tchangeover_progressive >= np.sum(summands))

In [ ]:
# get the system cost (average unit cost)
sys_cost = (np.sum(cholding) + 1e3*np.sum(mmax)*units('1/count'))/Qsys*units('count') + 1e-3*np.sum([l.L for l in sys_lines])/units('count')
# sys_cost = (np.sum(cholding) + np.sum(mmax)*units('1/count'))/Qsys*units('count')

# add some slight pressure on changeover times
# sys_cost = sys_cost + 1e-5/units('hr')*(Tchangeover_flowout + Tchangeover_progressive)


In [ ]:
# make system
sys = Model(sys_cost, [p1_line, p2_line, *const])

In [ ]:
# add a calculation for overall system usage (production time to total time)

nu_prod = Variable('\\nu_{production}', evalfn=lambda v: np.sum(v(T))*v(nperiods)/v(Thorizon))
sys.unique_varkeys = set([nu_prod.key])

In [ ]:
# delete rate substituions for products
try:
    del sys.substitutions[p1_line.lam]
except KeyError:
    pass
try:
    del sys.substitutions[p2_line.lam]
except KeyError:
    pass
# sys.substitutions.pop(p1_line.lam, None)
# sys.substitutions.pop(p2_line.lam, None)


# update system inputs
## horizon
sys.substitutions[Thorizon] = 1000*units('hr')
## quantity
sys.substitutions[Qsys] = 10E3
## average unit cost
sys.substitutions[davg] = 1
## holding rate
sys.substitutions[h] = 5e-2

## changeover times [mins]
# sys.substitutions[tci[0]] = 480     # at 480 changeover, the flow-out is preferred
# sys.substitutions[tci[1]] = 480
sys.substitutions[tci[0]] = 45
sys.substitutions[tci[1]] = 45

## ratios
sys.substitutions[X[0]] = 0.3
sys.substitutions[X[1]] = 0.7

In [ ]:
if select_best_case:
    print(sys.localsolve(solver='cvxopt').table())
else:
    print(sys.solve(solver='cvxopt').table())

Starting a sequence of GP solves
 for 3 free variables
  in 1 locally-GP constraints
  and for 60 free variables
       in 68 posynomial inequalities.
Solving took 0.601 seconds and 6 GP solves.

Optimal Cost
------------
 1.604

Warnings
Unexpectedly Tight Constraints
------------------------------
+0.085 : T[1] >= 2·FabLine93.L·FabLine93.W
 +0.03 : T[0] >= 2·FabLine92.L·FabLine92.W

Free Variables
--------------
            N_{periods} : 14.15                   [count]    number of production periods
   T_{change, flow-out} : 34.06                   [hr]       full flow-out changeover
T_{change, progressive} : 5.538                   [hr]       progressive changeover
         T_{changeover} : 5.539                   [hr]       total changeover time per period
              \max{t_c} : 0.75                    [hr]       path max changeover
       \nu_{production} : 0.9216
                      T : [ 19.1      46.1     ]  [hr]       production time per period
            c_{holding} : 

In [ ]:
sys.solution['variables'](nu_prod)*100.0

92.161792569084 <Unit('dimensionless')>

In [ ]:
print(sys.solution['variables'](select_t))

[1.80205452e-05 1.00009050e+00] dimensionless


In [ ]:
for l in sys_lines:
    for c in l.cells:
        print(sys.solution['variables'](c.rho))

0.3321719098626679 dimensionless
0.9609797013906447 dimensionless
0.9626402114580774 dimensionless
0.9283121936643483 dimensionless


In [ ]:
sys

<gpkit.Model object containing 20 top-level constraint(s) and 87 variable(s)>

In [ ]:
print(sys.localsolve(solver='cvxopt').table())

Starting a sequence of GP solves
 for 3 free variables
  in 1 locally-GP constraints
  and for 60 free variables
       in 67 posynomial inequalities.
Solving took 1.51 seconds and 19 GP solves.

Optimal Cost
------------
 0.1895

Warnings
Unexpectedly Tight Constraints
------------------------------
  +0.9 : T[1] >= 2·FabLine17.L·FabLine17.W
 +0.33 : T[0] >= 2·FabLine16.L·FabLine16.W

Free Variables
--------------
            N_{periods} : 11.85                   [count]    number of production periods
   T_{change, flow-out} : 32.28                   [hr]       full flow-out changeover
T_{change, progressive} : 21.83                   [hr]       progressive changeover
         T_{changeover} : 21.84                   [hr]       total changeover time per period
              \max{t_c} : 0.5                     [hr]       path max changeover
                      T : [ 18.2      44.4     ]  [hr]       production time per period
            c_{holding} : [ 677       1.19e+03 ]          

In [ ]:
sys.solution['warnings']

{'Unexpectedly Tight Constraints': [('Constraint [ T[0] [hr]... >= 2·FabLine6.L·FabLine6.W [count·hr]... ) is not loose: it has a sensitivity of +0.2799. (Allowable sensitivity: 1e-05)',
   gpkit.PosynomialInequality(T[0] >= 2·FabLine6.L·FabLine6.W)),
  ('Constraint [ T[1] [hr]... >= 2·FabLine7.L·FabLine7.W [count·hr]... ) is not loose: it has a sensitivity of +0.668. (Allowable sensitivity: 1e-05)',
   gpkit.PosynomialInequality(T[1] >= 2·FabLine7.L·FabLine7.W))]}

## Changeover in Concurrent Multiproduct

In concurrent multiproduct, there still may be changeovers which will be added to the cycle time through the cell. 

The overall probability that there will be a changeover is based on the portion of the product in each cell $x_i$. The changeover time between products in a cell is the same for all products $t_{changeover}$. For a given product $i$, the likelihood that there was a different product directly proceeding it is simply the compliment of $x_i$
$$ (1-x_i) $$

For an individual product $i$, the expected changeover is 
$$ t_{changeover}(1-x_i) $$


__Question__
- would there be a way to model dedicating workstations to specific products (as to avoid changeover)?
- probably the best way to capture that is to move it to a non-shared resource!


#### TODO
- will need to figure out the contribution of the changeover to the variabiilty of the process times. Should we be using a `SerialProcess` object to add the changeover to the flow time?
- along this same logic, does there need to be a `SerialProcess` that also describes the waiting for the blocked downstream cell? (That way we could feed back the departure variability of the downstream cell to this process)

__NOTE__
- for product variants, there can be no changeover. This is one of the hallamrks of a variants vs another product

## GPX Model Development

### Concurrent with changeover time

In [1]:
from gpx.multiclass.mccell import MCell

ImportError: No module named 'gpkit'